In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i = PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [12]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i
1,4326,0.5758,4326,0.328181,2681,0.097191
2,4362,0.5781,2543,0.335492,2582,0.121122
3,4332,0.5862,4316,0.346705,1066,0.121128


In [13]:
#return the total power of node i.
def TPN(i):
    PI_ofi = 1/Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    n1=TC_ofi /Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n2=PI_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    n3=CC_ofi/Tr_centrality_measure.TC_of(i)+Graph_Entropy_measure.PI_of(i)+Constraint_Coefficient.CC_of(i)
    #equation []
    #TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)    
    return TPN_i

In [14]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [15]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1])
TPN_Rank

[[2582, 3.070200234132779],
 [2681, 3.1981005824046567],
 [4316, 3.596257312525794],
 [2543, 3.597980580133207],
 [4332, 3.600447832656697],
 [4326, 3.609388143440632],
 [4362, 3.798341324799281],
 [1066, 6.797637886534841]]

In [16]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:2])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(Top_Five)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')
TPN_table

   nodes_TPN     TPN_i
1       2582  3.070200
2       2681  3.198101
3       4316  3.596257


,nodes_TPN,TPN_i
0,2582,3.070200
1,2681,3.198101
2,4316,3.596257
3,2543,3.597981
4,4332,3.600448
5,4326,3.609388
6,4362,3.798341
7,1066,6.797638


In [17]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_FiveR = pd.DataFrame(Propose_Result_table.loc[0:4])
Top_FiveR.index = Top_FiveR.index + 1
#convert column nodes_TPN to int
Top_FiveR.nodes_TPN= Top_FiveR.nodes_TPN.astype(int)
ranking_table = pd.concat([Top_FiveR['nodes_TC'], Top_FiveR['nodes_PI'], Top_FiveR['nodes_CC'],Top_FiveR['nodes_TPN']], axis=1, sort=False)
print(ranking_table)
#Propose_Table.to_csv('result/Propose_Table_power_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC  nodes_PI  nodes_CC  nodes_TPN
1      4326      4326      2681       2582
2      4362      2543      2582       2681
3      4332      4316      1066       4316
4      4360      4360      3288       2543
5      4352      2624      2542       4332


In [18]:
# calculate the kendall's correlation between two variables
from numpy.random import rand
from numpy.random import seed
from scipy.stats import kendalltau

# prepare data
data1 = Top_FiveR['nodes_TPN']
data2 = data1 + Top_FiveR['nodes_CC']+ Top_FiveR['nodes_PI']+ Top_FiveR['nodes_TC']
# calculate kendall's correlation
coef, p = kendalltau(data1, data2)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)


Kendall correlation coefficient: -0.400
Samples are uncorrelated (fail to reject H0) p=0.327


In [19]:
# Importing required libraries
import researchpy as rp
from scipy import stats
#The casewise method returns all information in 3 dataframes
#corr_type, corr_matrix, corr_ps=>
corr_type, corr_matrix, corr_ps = rp.corr_case(Top_FiveR[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])
#PAIRWISEcorrelation
corr_pair = rp.corr_pair(Top_FiveR[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])